In [3]:
import os
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import polars as pd
from datetime import datetime

In [4]:
def closePoup(d=None):
    try:
        elem = d.find_element(By.XPATH,'/html/body/div[5]/div/div/section/button')
        elem.click()
    except:
        pass

def showMore(d=None):
    paths = ["/html/body/main/section[1]/div/div/section[1]/div/div/section/button[1]"
            ,"/html/body/main/section[1]/div/div/section[1]/div/div[2]/section/button[1]"]

    for path in paths:
        
        try:
            elem = d.find_element(By.XPATH,path)
            elem.click()
            return 1
        except:
            pass

def prevWindow(d=None):
    try:
        d.execute_script("window.history.go(-1)")    
        time.sleep(5)
    except:
        pass

In [27]:
position = "Data%20Engineer"

#Input the job location you're looking for
locations = ["Hong Kong","Riyadh", "Dubai", "Abu Dhabi"]
geo_ids = ["102817007","100264288", "106204383", "103720977"]

#Filter job posting date
period = "2592000" #past month
# period = "604800" #past week
# period = "86400" #past 24 hours


#Filter experience level
#experience ="&f_E=1" #Internship
#experience ="&f_E=2" #Entry-level
#experience ="&f_E=3" #Associate
#experience ="&f_E=2%2C3" #Entry-level + Associate
experience ="&f_E=4" #Mid-Senior level

# Initialize arrrays for different columns
loc_row = []
job_title = []
job_comp = []
job_function = []
job_seniority_level = []
job_industry = []
description = []
disc_list = []

In [29]:

#Step 2: Web scraping (Use public Linkedin search)
#--------------------------------------------------------------------


for i in range(0,len(locations)):
    # driver = webdriver.Chrome()
    service = ChromeService()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    location = locations[i].replace(' ', "%20")
    geo_id = geo_ids[i]

    url = f"https://www.linkedin.com/jobs/search/?keywords={position}&location={location}&geoId={geo_id}&f_TPR=r{period}{experience}&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0%27"
    
    driver.get(url)

    time.sleep(2)

    closePoup(driver)
    
    last_height = driver.execute_script('return document.body.scrollHeight')
    for i in range(3):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(5)
        new_height = driver.execute_script('return document.body.scrollHeight')
        last_height = new_height

    
    the_list = driver.find_element(By.CLASS_NAME, 'jobs-search__results-list')
    jobs = the_list.find_elements(By.TAG_NAME,'li') # return a list of web elements
    
    for job in range(1,len(jobs)+1):
        
        try:
            job_click = driver.find_element(By.XPATH, f'//*[@id="main-content"]/section[2]/ul/li[{job}]/div/a').click()
            closePoup(driver)
            showMore(driver)
            time.sleep(5)
            
            loc_row.append(location.replace('%20',' '))
            try:   
                job_title0 = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h1').text
                job_title.append(job_title0)
                
            except:
                job_title.append('NIL')
                
            try:             
                job_comp0 = driver.find_element(By.XPATH, '/html/body/main/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a').text
                job_comp.append(job_comp0)
            except:
                job_comp.append('NIL')
                
            try:             
                job_function0 = driver.find_element(By.XPATH, '/html/body/main/section[1]/div/div/section[1]/div/ul/li[3]/span').text
                job_function.append(job_function0)
            except:
                job_function.append('NIL')
                
            try:            
                    
                job_seniority_level0 = driver.find_element(By.XPATH, '/html/body/main/section[1]/div/div/section[1]/div/ul/li[1]/span').text
                job_seniority_level.append(job_seniority_level0)
            except:
                job_seniority_level.append('NIL')
            
            try:            
                    
                job_industry0 = driver.find_element(By.XPATH, '/html/body/main/section[1]/div/div/section[1]/div/ul/li[4]/span').text
                job_industry.append(job_industry0)
            except:
                job_industry.append('NIL')
    
            try:
                # job_desc = driver.find_element(By.XPATH, '//*[@id="job-details"]').text
                job_desc = driver.find_elements(By.XPATH,'/html/body/main/section[1]/div/div/section[1]/div/div/section/div')
                description.append(job_desc[0].text.replace('\n', ' '))
                # disc_list.append(job_desc)
            except:
                # disc_list.append("dd")
                description.append("dd")

            prevWindow(driver)
        except:
            pass

    
    driver.quit()
    

df = pd.DataFrame({
'Location': loc_row,
'Company': job_comp,
'Position': job_title,
'Level': job_seniority_level,
'Function': job_function,
'Industry':job_industry,
'Job Description': description
})

filename = 'LinkedinJobs_'+str(datetime.now())[:19].replace(':','').replace('-','').replace(' ','')+'.json'

df.write_json(filename)